## Computing the Minimum Variance Frontier: Unconstrained

An investor considers an investment opportunity set that consists of two risky assets. One asset has an expected return of 8\% and a volatility of 16% while the other has an expected return of only 3% and a volatility of 4%. Both asset returns share a correlation of 20%. 

**Instruction:** Use the same notation as in the lecture and revision manual.

(i) Compute the weights, expected returns and volatiliy of minimum variance portfolios. Use the following specifications
- compute the frontier for 400 target (expected) returns in the range of 0 and 20%

(ii) Compute the weights, expected return and volatility of the global minimum variance portfolio

(iii) Assume a risk-free rate of 2%. Compute the expected return, volatility and portfolio weights of the tangency portfolio

(iv) Create one graph displaying the expected return and volatility of the investment opportunity set, the minimum variance frontier, the tangency portfolio, the global minimum variance portfolio, the capital allocation line, the risk-free rate.


In [14]:
# needed libraries
import numpy as np
from operator import itemgetter

In [4]:
# constants
mu_1 = 0.08
mu_2 = 0.03
vol_1 = 0.16
vol_2 = 0.04
corr= 0.2
cov = vol_1 * vol_2 * corr
mu = np.array([mu_1,mu_2])
rf = 0.02

# covmatrix
cov_m = np.matrix([[vol_1*vol_1,cov], [cov,vol_2*vol_2]])

# reshape as colum vector
mu = mu.reshape((-1, 1))

# inverse of matrix
conv_m_invert = np.linalg.inv(cov_m)

# dimension
n = np.shape(conv_m_invert)[1]

# array with ones: (1,...,1): column vector with
one_vec = np.array(np.repeat(1,n))

# reshape as colum vector
one_vec = one_vec.reshape((-1, 1))

# mu range for portfolios
mu_range = np.linspace(start=0,stop=0.2,num =400)
mu_200 = mu_range[200-1]
mu_400 = mu_range[400-1]

# print(mu)
# print(conv_m_invert)
# print(one_vec)


In [26]:
# task i
def get_mvf_properties(mu,cov_m,conv_m_invert,one_vec,mu_mv):
    """
    This function calculates the portfolis localated at the minimum-variance-frontier.

    :param mu: column vector with (mu_1,....,mu_n)
    :param cov_m: Covariance matrix
    :param conv_m_invert: inverted covariance matrix
    :param one_vec: column vector with ones
    :param mu_mv: fixed given return
    :returns:weigths_gmvp,mu_gmvp, vol_gmvp
    """

    # 1x2 * 2x2 * 2x1 = 1x1 => skalar
    x = mu.T * conv_m_invert * mu
    x = float(x)

    # 1x2 * 2x2 * 2x1 = 1x1 => skalar
    y = one_vec.T * conv_m_invert * mu
    y = float(y)

    # 1x2 * 2x2 * 2x1 = 1x1 => skalar
    z = one_vec.T * conv_m_invert * one_vec
    z = float(z)

    # multiplication of scalar values
    d = x*z - y*y
    d = float(d)

    # skalar * (skalar * 2x2 * 2x1 - skalar * 2x2 * 2x1) = 2x1
    g = 1/d *(x*conv_m_invert*one_vec - y*conv_m_invert * mu)

    # skalar * (skalar * 2x2 * 2x1 - skalar * 2x2 * 2x1) = 2x1
    h = 1/d * (z*conv_m_invert*mu - y*conv_m_invert*one_vec)

    # 2x1 * 1x1 = 2x1
    h_mu = h * mu_mv

    # calculate the weights of the mvf
    weigths_mvf = g + h_mu

    # print("weights:")
    # print(round(float(weigths_mvf[0]*100),2))
    # print(round(float(weigths_mvf[1]*100),2))

    # calculate the returns
    ret_i = weigths_mvf[0] * mu[0] + weigths_mvf[1] * mu[1]

    # print("Mu:")
    # print(round(float(ret_i*100),2))

    # calcualte the sd
    vol_i = weigths_mvf.T * cov_m * weigths_mvf
    vol_i = np.sqrt(vol_i)

    # print("SD:")
    # print(round(float(vol_i*100),2))

    return weigths_mvf

weigths_mvf_200 = get_mvf_properties(mu,cov_m,conv_m_invert,one_vec,mu_200)

weigths_mvf_400 = get_mvf_properties(mu,cov_m,conv_m_invert,one_vec,mu_400)

weights:
139.5
-39.5
Mu:
9.97
SD:
22.06
weights:
340.0
-240.0
Mu:
20.0
SD:
53.32


In [25]:
# task ii
def get_gmv_properties(mu,conv_m_invert,one_vec):
    """
    This function calculates the global minimum variance portfolio.

    :param mu: column vector with (mu_1,....,mu_n)
    :param conv_m_invert: inverted covariance matrix
    :param one_vec: column vector with ones
    :returns: weigths_gmvp, mu_gmvp, vol_gmvp
    """

    # calculate the weights of the gmv: 2x2 * 2x1 / 1x2 * 2x2 * 2x1
    weigths_gmvp = (conv_m_invert*one_vec) / (one_vec.T*conv_m_invert*one_vec)

    print("weights:")
    print(round(float(weigths_gmvp[0]*100),2))
    print(round(float(weigths_gmvp[1]*100),2))

    # calculate the return of the gmv:
    mu_gmvp = (mu.T*conv_m_invert*one_vec) / (one_vec.T*conv_m_invert*one_vec)

    print("Mu:")
    print(round(float(mu_gmvp*100),2))

    # calculate the votatiltiy of the gmv:
    vol_gmvp = 1/(one_vec.T*conv_m_invert*one_vec)

    # taking the standard deviation
    vol_gmvp = np.sqrt(vol_gmvp)

    print("SD:")
    print(round(float(vol_gmvp*100),2))

    return weigths_gmvp,mu_gmvp, vol_gmvp

weigths_gmvp,mu_gmvp, vol_gmvp = get_gmv_properties(mu,conv_m_invert,one_vec)

weights:
1.3
98.7
Mu:
3.06
SD:
3.99


In [49]:
# task iii
def get_tp_properties_alternative(rf,mu,cov_m,conv_m_invert,one_vec):
    """
    This functions calculates some tangency portfolio properties: schema from Investments.

    :param rf: risk free rate
    :param mu: returns
    :param cov_m: covariance matrix
    :param conv_m_invert: inverted covariance matrix
    :param one_vec: vector containing the ones
    :return: weigths_tp,mu_tp, vol_tp, sharpe_ratio
    """
    # 1x2 * 2x2 * 2x1 = 1x1 => skalar
    # B
    x = mu.T * conv_m_invert * mu
    x = float(x)

    # # 1x2 * 2x2 * 2x1 = 1x1 => skalar
    # A
    y = one_vec.T * conv_m_invert * mu
    y = float(y)

    # # 1x2 * 2x2 * 2x1 = 1x1 => skalar
    # C
    z = one_vec.T * conv_m_invert * one_vec
    z = float(z)

    w = conv_m_invert * (0.1 - rf) / (x - 2*y*rf + z* rf*rf) * (mu - rf*one_vec)
    weigths_tp = w/sum(w)

    print("weights:")
    print(round(float(weigths_tp[0]*100),2))
    print(round(float(weigths_tp[1]*100),2))

    # compute the return of the TP
    mu_tp = weigths_tp[0] * mu_1 + weigths_tp[1] * mu_2
    print("Mu:")
    print(round(float(mu_tp*100),2))

    # compute the volatility of the TP
    vol_tp = weigths_tp.T * cov_m * weigths_tp
    vol_tp = np.sqrt(vol_tp)
    print("SD:")
    print(round(float(vol_tp*100),2))

    # calculate the sharpe ratio on top
    sharpe_ratio = (mu_tp - rf) / vol_tp
    print("Sharpe ratio:")
    print(round(float(sharpe_ratio),2))

    return weigths_tp,mu_tp, vol_tp, sharpe_ratio

# weigths_tp,mu_tp, vol_tp,sharpe_ratio = get_tp_properties_alternative(rf,mu,cov_m,conv_m_invert,one_vec)

weights:
31.71
68.29
Mu:
4.59
SD:
6.22
Sharpe ratio:
0.42


In [ ]:
# task iii
def get_tp_properties_investments(rf,mu_1,mu_2,cov_m,conv_m_invert):
    """
    This functions calculates some tangency portfolio properties: schema from Investments.

    :param rf: risk free interest rate
    :param mu_1: mu of asset 1
    :param mu_2: mu of asset 2
    :param cov_m: covariance matrix
    :param conv_m_invert: inverted covariance matrix
    :return: weigths_tp,mu_tp, vol_tp, sharpe_ratio

    """

    # vecotor of adjusted returns
    mu_adj = np.array([mu_1-rf,mu_2-rf])
    mu_adj = mu_adj.reshape((-1, 1))

    # weights for random portfolio a
    w_a = conv_m_invert * mu_adj

    # weights of tangency portfolio
    weigths_tp = w_a/sum(w_a)
    print("weights:")
    print(round(float(weigths_tp[0]),2))
    print(round(float(weigths_tp[1]),2))

    # compute the return of the TP
    mu_tp = weigths_tp[0] * mu_1 + weigths_tp[1] * mu_2
    print("Mu:")
    print(round(float(mu_tp*100),2))

    # compute the volatility of the TP
    vol_tp = weigths_tp.T * cov_m * weigths_tp
    vol_tp = np.sqrt(vol_tp)
    print("SD:")
    print(round(float(vol_tp*100),2))

    # calculate the sharpe ratio on top
    sharpe_ratio = (mu_tp - rf) / vol_tp
    print("Sharpe ratio:")
    print(round(float(sharpe_ratio),2))

    return weigths_tp,mu_tp, vol_tp, sharpe_ratio

# weigths_tp,mu_tp, vol_tp,sharpe_ratio = get_tp_properties_investments(rf,mu_1,mu_2,cov_m,conv_m_invert)


In [23]:
def x(mu_fixed):
    weigths = get_mvf_properties(mu,cov_m,conv_m_invert,one_vec,mu_fixed)
    return  weigths

def f(mu_fixed):
    weigths = get_mvf_properties(mu,cov_m,conv_m_invert,one_vec,mu_fixed)
    ret = weigths[0] * mu[0] + weigths[1] * mu[1]
    vol = weigths.T * cov_m * weigths
    vol = np.sqrt(vol)
    return  (ret-rf) / vol

save_array = []
for i in mu_range:
    x_value = x(i)
    x_1 = float(x_value[0])
    x_2 = float(x_value[1])
    y = float(f(i))
    save_array.append((x_1,x_2,y))

# print(save_array)
# results:
w_opt= max(save_array,key=itemgetter(2)) #
print(w_opt)
w_opt_1 = w_opt[0]
w_opt_2 = w_opt[1]
weigths_tp = np.array([w_opt_1,w_opt_2])
weigths_tp = weigths_tp.reshape((-1, 1))
sharpe_ratio_opt = w_opt[2]
print("optimal weights:")
print(round(float(w_opt_1*100),2))
print(round(float(w_opt_2*100),2))

print("optimal sharpe ratio:")
print(round(float(sharpe_ratio_opt),2))

# calculate the return
mu_tp = w_opt_1 * mu_1 + w_opt_2* mu_2
print("MU:")
print(round(float(mu_tp*100),2))

# calculate the variance
vol_tp = weigths_tp.T * cov_m * weigths_tp

# taking the standard deviation
vol_tp = np.sqrt(vol_tp)

print("SD:")
print(round(float(vol_tp*100),2))





(0.31228070175438594, 0.6877192982456122, 0.41534981220896405)
optimal weights:
31.23
68.77
optimal sharpe ratio:
0.42
MU:
4.56
SD:
6.17
